<a href="https://colab.research.google.com/github/marcelofschiavo/ds-cookbook/blob/main/01_Coleta_e_Limpeza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. Coleta e Limpeza de Dados (SQL & Pandas)

**Objetivo:** Nenhum modelo de Machine Learning pode gerar valor a partir de dados ruins ("Garbage In, Garbage Out").
Aqui estão as "receitas" fundamentais para buscar, carregar, inspecionar e limpar dados brutos do mundo real, usando as duas ferramentas essenciais: SQL (para bancos de dados) e Pandas (para manipulação em Python).

## 1. SQL  

### Receita 1.1: `SELECT`, `FROM`, `WHERE`

* **🧠 Intuição:** É o básico do pedido. "Ei, banco de dados (`FROM vendas`), me traga (`SELECT`) a lista de `produto` e `quantidade` vendida, mas *apenas* (`WHERE`) da categoria `Eletrônicos`."
* **🎓 Definição Técnica:** `SELECT` define as colunas (projeção) a serem retornadas. `FROM` especifica a tabela fonte dos dados. `WHERE` aplica um filtro nas linhas (seleção) com base em uma condição lógica.
* **🍳 Receita:**
    ```sql
    SELECT
        produto,
        quantidade
    FROM
        vendas
    WHERE
        categoria = 'Eletrônicos';
    ```
* **📊 Resultado:** Uma tabela contendo apenas os produtos e quantidades da categoria 'Eletrônicos'. As outras categorias e colunas são omitidas.
    ```
    | produto | quantidade |
    | :------ | :--------- |
    | Laptop  | 1          |
    | Teclado | 2          |
    | Monitor | 1          |
    | Mouse   | 3          |
    ```

### Receita 1.2: `GROUP BY`

* **🧠 Intuição:** "Agora, quero saber o *total* vendido de cada categoria. Primeiro, faça 'montinhos' (`GROUP BY`) por `categoria`. Depois, some (`SUM`) a `quantidade` de cada montinho."
* **🎓 Definição Técnica:** Agrupa linhas que compartilham o mesmo valor na coluna especificada (`categoria`). É usado em conjunto com funções de agregação (`SUM`, `AVG`, `COUNT`, `MIN`, `MAX`) que operam sobre cada grupo formado. O `AS` renomeia a coluna resultante.
* **🍳 Receita:**
    ```sql
    SELECT
        categoria,
        SUM(quantidade) AS total_vendido,
        AVG(preco_unitario) AS preco_medio
    FROM
        vendas
    GROUP BY
        categoria;
    ```
* **📊 Resultado:** Uma tabela resumida, mostrando o total de itens vendidos e o preço unitário médio para cada categoria. Note que a linha da 'Mesa' (quantidade NULL) pode ser ignorada pelo `SUM` dependendo da configuração do banco.
    ```
    | categoria   | total_vendido | preco_medio |
    | :---------- | :------------ | :---------- |
    | Eletrônicos | 7             | 1232.50     |
    | Casa        | 10            | 25.00       |
    | Móveis      | 1             | 700.00      |
    ```
    *(O preco_medio de Móveis pode variar se o NULL for incluído ou não no AVG)*

### Receita 1.3: `JOIN`

* **🧠 Intuição:** "Eu tenho a lista de vendas (com `id_cliente`) e a lista de clientes (com `id_cliente` e `nome_cliente`). Quero 'colar' as duas usando o `id_cliente` para saber *quem* comprou *o quê*."
* **🎓 Definição Técnica:** Combina linhas de duas ou mais tabelas (`vendas` e `clientes`) com base em uma coluna relacionada (`id_cliente`). `INNER JOIN` retorna apenas as linhas onde a chave existe em *ambas* as tabelas. A cláusula `ON` especifica a condição de junção (igualdade entre a Chave Estrangeira `v.id_cliente` e a Chave Primária `c.id_cliente`). `v` e `c` são apelidos (aliases) para as tabelas.
* **🍳 Receita:**
    ```sql
    SELECT
        v.produto,
        v.quantidade,
        c.nome_cliente,
        c.cidade
    FROM
        vendas v
    INNER JOIN
        clientes c ON v.id_cliente = c.id_cliente;
    ```
* **📊 Resultado:** Uma tabela combinada mostrando o produto, quantidade, nome do cliente e cidade. A venda da Mesa (cliente 104) aparecerá aqui porque ele existe na tabela `clientes` do nosso exemplo Python.
    ```
    | produto       | quantidade | nome_cliente | cidade      |
    | :------------ | :--------- | :----------- | :---------- |
    | Laptop        | 1          | Ana Silva    | São Paulo   |
    | Teclado       | 2          | Bruno Lima   | Rio Janeiro |
    | Caneca        | 10         | Ana Silva    | São Paulo   |
    | Monitor       | 1          | Carla Dias   | São Paulo   |
    | Cadeira Gamer | 1          | Bruno Lima   | Rio Janeiro |
    | Mouse         | 3          | Ana Silva    | São Paulo   |
    | Mesa de Escrit.| NaN        | Daniel Reis  | B. Horizonte|
    ```

## 2. Pandas (Python Data Analysis Library)

Faz quase tudo que o SQL faz, mas diretamente na memória do Python, com objetos chamados DataFrames.
O código abaixo cria nossos dois DataFrames principais na memória:

In [5]:
import pandas as pd
import numpy as np # Usaremos para criar um valor NaN

# Criando o DataFrame 'vendas' diretamente no Python (listas dentro de um dicionário)
data_vendas = {
    'id_venda': [1, 2, 3, 4, 5, 6, 7],
    'produto': ['Laptop', 'Teclado', 'Caneca', 'Monitor', 'Cadeira Gamer', 'Mouse', 'Mesa de Escrit.'],
    'categoria': ['Eletrônicos', 'Eletrônicos', 'Casa', 'Eletrônicos', 'Móveis', 'Eletrônicos', 'Móveis'],
    'quantidade': [1, 2, 10, 1, 1, 3, np.nan], # Adicionando NaN (NULL)
    'preco_unitario': [3500.00, 150.00, 25.00, 1200.00, 950.00, 80.00, 450.00],
    'id_cliente': [101, 102, 101, 103, 102, 101, 104],
    'data_venda': pd.to_datetime(['2025-10-20', '2025-10-21', '2025-10-21', '2025-10-22', '2025-10-23', '2025-10-24', '2025-10-25']) # Convertendo para data
}
df_vendas = pd.DataFrame(data_vendas)

# Criando o DataFrame 'clientes'
data_clientes = {
    'id_cliente': [101, 102, 103, 104],
    'nome_cliente': ['Ana Silva', 'Bruno Lima', 'Carla Dias', 'Daniel Reis'],
    'cidade': ['São Paulo', 'Rio Janeiro', 'São Paulo', 'B. Horizonte']
}
df_clientes = pd.DataFrame(data_clientes)

print("DataFrames criados:")
print("--- Vendas ---")
print(df_vendas)
print("\n--- Clientes ---")
print(df_clientes)

DataFrames criados:
--- Vendas ---
   id_venda          produto    categoria  quantidade  preco_unitario  \
0         1           Laptop  Eletrônicos         1.0          3500.0   
1         2          Teclado  Eletrônicos         2.0           150.0   
2         3           Caneca         Casa        10.0            25.0   
3         4          Monitor  Eletrônicos         1.0          1200.0   
4         5    Cadeira Gamer       Móveis         1.0           950.0   
5         6            Mouse  Eletrônicos         3.0            80.0   
6         7  Mesa de Escrit.       Móveis         NaN           450.0   

   id_cliente data_venda  
0         101 2025-10-20  
1         102 2025-10-21  
2         101 2025-10-21  
3         103 2025-10-22  
4         102 2025-10-23  
5         101 2025-10-24  
6         104 2025-10-25  

--- Clientes ---
   id_cliente nome_cliente        cidade
0         101    Ana Silva     São Paulo
1         102   Bruno Lima   Rio Janeiro
2         103   Carla D

### Receita 2.1: Ler um CSV (`pd.read_csv`)

* **🧠 Intuição:** "Dizer ao Python: 'Abra este arquivo (`vendas.csv`) que parece uma planilha e coloque os dados na nossa mesa de trabalho (`df_vendas`)'."
* **🎓 Definição Técnica:** Função do Pandas para ler arquivos de texto delimitados (como CSV) em um objeto DataFrame. Possui diversos parâmetros para lidar com diferentes formatos (separador `sep`, cabeçalho `header`, codificação `encoding`, tratamento de erros `on_bad_lines`, etc.).
* **🍳 Receita:**

In [12]:
# 1. Primeiro, vamos SALVAR nosso df_vendas em um arquivo CSV
#    Usamos index=False para não salvar o índice 0,1,2... como uma coluna
try:
    df_vendas.to_csv('vendas_para_teste.csv', index=False, sep=';', encoding='utf-8')
    print("Arquivo 'vendas_para_teste.csv' salvo com sucesso (separador=';').\n")

    # 2. Agora, vamos LER o arquivo que acabamos de criar.
    #    Esta é a receita executável do pd.read_csv()
    df_lido = pd.read_csv('vendas_para_teste.csv', sep=';')

    print("--- Arquivo CSV lido de volta para a variável 'df_lido': ---")
    print(df_lido.head()) # .head() mostra as 5 primeiras linhas

except Exception as e:
    print(f"Ocorreu um erro ao salvar/ler o arquivo: {e}")
    print("Verifique as permissões da pasta.")

Arquivo 'vendas_para_teste.csv' salvo com sucesso (separador=';').

--- Arquivo CSV lido de volta para a variável 'df_lido': ---
   id_venda        produto    categoria  quantidade  preco_unitario  \
0         1         Laptop  Eletrônicos         1.0          3500.0   
1         2        Teclado  Eletrônicos         2.0           150.0   
2         3         Caneca         Casa        10.0            25.0   
3         4        Monitor  Eletrônicos         1.0          1200.0   
4         5  Cadeira Gamer       Móveis         1.0           950.0   

   id_cliente  data_venda  
0         101  2025-10-20  
1         102  2025-10-21  
2         101  2025-10-21  
3         103  2025-10-22  
4         102  2025-10-23  


* **📊 Resultado:** Se o arquivo 'vendas.csv' existisse no diretório, a variável `df_vendas_lido` conteria um DataFrame idêntico ao `df_vendas` que criamos manualmente. `.head()` é um método que exibe as primeiras 5 linhas do DataFrame, útil para uma verificação rápida.

### Receita 2.2: Filtrar (`df[...]`) - Equivalente ao `WHERE`

* **🧠 Intuição:** "Da nossa mesa de trabalho (`df_vendas`), me mostre *apenas* as linhas onde a coluna `categoria` é igual a `'Eletrônicos'`. Ou talvez, onde a `quantidade` é maior que 5."
* **🎓 Definição Técnica:** Utiliza **Boolean Masking**. A expressão condicional (ex: `df_vendas['categoria'] == 'Eletrônicos'`) gera uma Série Pandas de valores `True` ou `False` (a máscara). Passar essa máscara dentro dos colchetes `df_vendas[...]` seleciona apenas as linhas onde a máscara é `True`. Condições múltiplas podem ser combinadas usando `&` (AND lógico) e `|` (OR lógico), agrupadas por parênteses devido à precedência de operadores.
* **🍳 Receita:**

In [13]:
# Filtro simples: Selecionar apenas a categoria 'Eletrônicos'
df_eletronicos = df_vendas[df_vendas['categoria'] == 'Eletrônicos']
print("--- Apenas Eletrônicos ---")
print(df_eletronicos)

# Filtro composto: Categoria 'Eletrônicos' E quantidade maior que 1
# (Note os parênteses obrigatórios em volta de cada condição)
df_eletronicos_multiplos = df_vendas[
    (df_vendas['categoria'] == 'Eletrônicos') & (df_vendas['quantidade'] > 1)
]
print("\n--- Eletrônicos com Quantidade > 1 ---")
print(df_eletronicos_multiplos)

# Filtro usando .isin() para múltiplos valores (o oposto de `WHERE categoria IN (...)`)
categorias_interesse = ['Casa', 'Móveis']
df_casa_moveis = df_vendas[df_vendas['categoria'].isin(categorias_interesse)]
print("\n--- Apenas Categorias 'Casa' ou 'Móveis' ---")
print(df_casa_moveis)

--- Apenas Eletrônicos ---
   id_venda  produto    categoria  quantidade  preco_unitario  id_cliente  \
0         1   Laptop  Eletrônicos         1.0          3500.0         101   
1         2  Teclado  Eletrônicos         2.0           150.0         102   
3         4  Monitor  Eletrônicos         1.0          1200.0         103   
5         6    Mouse  Eletrônicos         3.0            80.0         101   

  data_venda  
0 2025-10-20  
1 2025-10-21  
3 2025-10-22  
5 2025-10-24  

--- Eletrônicos com Quantidade > 1 ---
   id_venda  produto    categoria  quantidade  preco_unitario  id_cliente  \
1         2  Teclado  Eletrônicos         2.0           150.0         102   
5         6    Mouse  Eletrônicos         3.0            80.0         101   

  data_venda  
1 2025-10-21  
5 2025-10-24  

--- Apenas Categorias 'Casa' ou 'Móveis' ---
   id_venda          produto categoria  quantidade  preco_unitario  \
2         3           Caneca      Casa        10.0            25.0   
4        

* **📊 Resultado:** `df_eletronicos` conterá um novo DataFrame apenas com as 4 linhas correspondentes a produtos eletrônicos. `df_eletronicos_multiplos` conterá apenas as 2 linhas de 'Teclado' e 'Mouse', que satisfazem ambas as condições. `df_casa_moveis` conterá as linhas de 'Caneca', 'Cadeira Gamer' e 'Mesa de Escrit.'. A filtragem é essencial para isolar subconjuntos de dados para análises específicas.

### Receita 2.3: Agrupar (`.groupby().agg()`) - Equivalente ao `GROUP BY`

* **🧠 Intuição:** "Vamos fazer 'montinhos' (`groupby`) por `categoria` na nossa mesa de trabalho. Depois, para cada montinho, vamos calcular (`agg`) várias coisas: o total vendido (`sum` da quantidade), o preço médio (`mean` do preço) e quantas vendas teve (`count`)."
* **🎓 Definição Técnica:** O método `.groupby()` cria um objeto `DataFrameGroupBy`, que representa os dados particionados pelos grupos especificados. Métodos de agregação (`.sum()`, `.mean()`, `.count()`, `.min()`, `.max()`, `.std()`, etc.) podem ser encadeados para calcular estatísticas por grupo. O método `.agg()` é mais flexível, permitindo aplicar múltiplas funções de agregação simultaneamente e renomear as colunas resultantes. Ele recebe um dicionário onde as chaves são os nomes das novas colunas e os valores são tuplas `('coluna_original', 'funcao_agregacao_string')` ou diretamente a função de agregação.
* **🍳 Receita:**

In [14]:
resumo_categoria_pd = df_vendas.groupby('categoria').agg(
    total_vendido=('quantidade', 'sum'),           # Soma as quantidades
    preco_medio=('preco_unitario', 'mean'),        # Calcula a média dos preços
    contagem_vendas=('id_venda', 'count'),         # Conta o número de vendas (linhas)
    preco_maximo=('preco_unitario', 'max')         # Encontra o preço unitário máximo
).reset_index() # Transforma o índice 'categoria' de volta em coluna

print("--- Resumo por Categoria (Pandas) ---")
print(resumo_categoria_pd)

--- Resumo por Categoria (Pandas) ---
     categoria  total_vendido  preco_medio  contagem_vendas  preco_maximo
0         Casa           10.0         25.0                1          25.0
1  Eletrônicos            7.0       1232.5                4        3500.0
2       Móveis            1.0        700.0                2         950.0


* **📊 Resultado:** Produz um novo DataFrame, onde cada linha representa uma `categoria`. As colunas mostram as estatísticas agregadas calculadas para cada grupo: a soma total de `quantidade` (ignorando o NaN), a média do `preco_unitario`, a contagem de `id_venda` (número de produtos naquela categoria) e o `preco_unitario` mais alto encontrado. O `.reset_index()` é usado para que `categoria` volte a ser uma coluna normal, facilitando manipulações posteriores.

### Receita 2.4: Juntar (`pd.merge()`) - Equivalente ao `JOIN`

* **🧠 Intuição:** "Vamos 'colar' (`merge`) a nossa mesa de vendas (`df_vendas`) com a mesa de clientes (`df_clientes`) usando o `id_cliente` como 'cola', para que possamos ver o nome e a cidade do cliente ao lado de cada venda."
* **🎓 Definição Técnica:** A função `pd.merge()` combina dois DataFrames com base em uma ou mais colunas comuns (chaves), similar às operações JOIN em SQL.
    * `on`: Especifica a(s) coluna(s) chave(s) para a junção. Se os nomes forem diferentes, usa-se `left_on` e `right_on`.
    * `how`: Define o tipo de junção:
        * `'inner'` (padrão): Retorna apenas linhas com chaves correspondentes em *ambos* DataFrames (equivalente a INNER JOIN).
        * `'left'`: Retorna todas as linhas do DataFrame da *esquerda* (`df_vendas`) e as correspondentes da direita (`df_clientes`). Se não houver correspondência, preenche com `NaN` (equivalente a LEFT JOIN).
        * `'right'`: O oposto do 'left'.
        * `'outer'`: Retorna todas as linhas de *ambos*, preenchendo com `NaN` onde não há correspondência.
* **🍳 Receita:**

In [15]:
# Junção padrão (INNER JOIN)
df_completo_inner = pd.merge(
    df_vendas,
    df_clientes,
    on='id_cliente',
    how='inner'
)
print("--- Tabela Combinada (INNER JOIN) ---")
print(df_completo_inner)

# Exemplo com LEFT JOIN (para incluir vendas mesmo sem cliente correspondente, se houvesse)
# df_completo_left = pd.merge(
#     df_vendas,
#     df_clientes,
#     on='id_cliente',
#     how='left'
# )
# print("\n--- Tabela Combinada (LEFT JOIN) ---")
# print(df_completo_left)

--- Tabela Combinada (INNER JOIN) ---
   id_venda          produto    categoria  quantidade  preco_unitario  \
0         1           Laptop  Eletrônicos         1.0          3500.0   
1         2          Teclado  Eletrônicos         2.0           150.0   
2         3           Caneca         Casa        10.0            25.0   
3         4          Monitor  Eletrônicos         1.0          1200.0   
4         5    Cadeira Gamer       Móveis         1.0           950.0   
5         6            Mouse  Eletrônicos         3.0            80.0   
6         7  Mesa de Escrit.       Móveis         NaN           450.0   

   id_cliente data_venda nome_cliente        cidade  
0         101 2025-10-20    Ana Silva     São Paulo  
1         102 2025-10-21   Bruno Lima   Rio Janeiro  
2         101 2025-10-21    Ana Silva     São Paulo  
3         103 2025-10-22   Carla Dias     São Paulo  
4         102 2025-10-23   Bruno Lima   Rio Janeiro  
5         101 2025-10-24    Ana Silva     São Paulo  

* **📊 Resultado:** `df_completo_inner` conterá um DataFrame com todas as colunas de `df_vendas` e `df_clientes` (exceto `id_cliente` duplicado). Como todos os `id_cliente` em `df_vendas` existem em `df_clientes` neste exemplo, o resultado do `inner` e do `left` seria o mesmo. O `merge` é fundamental para enriquecer os dados de uma tabela com informações de outra.

### Receita 2.5: Lidar com Dados Nulos (`.isnull()`, `.fillna()`, `.dropna()`)

* **🧠 Intuição:** "Ops! A 'quantidade' da Mesa de Escritório está vazia (`NaN`, que é o 'NULL' do Pandas). Isso vai dar problema nos cálculos. O que fazemos? (1) Jogamos a linha inteira da Mesa fora (`dropna`)? Ou (2) Tentamos 'adivinhar' um valor para colocar ali (`fillna`), como a quantidade '1' (que é a mais comum para móveis)?"
* **🎓 Definição Técnica:** Dados ausentes (`NaN` - Not a Number) são valores indefinidos que precisam ser tratados antes da análise ou modelagem.
    * `.isnull()`: Retorna uma máscara booleana (`True` onde for `NaN`). Encadeado com `.sum()` (`df.isnull().sum()`), conta os `NaN`s por coluna.
    * `.dropna()`: Remove linhas (padrão, `axis=0`) ou colunas (`axis=1`) que contêm pelo menos um `NaN`. O argumento `subset` permite especificar colunas onde procurar por `NaN`s. É uma abordagem simples, mas pode levar à perda significativa de dados.
    * `.fillna()`: Preenche os `NaN`s com um valor especificado. Pode ser um valor escalar (ex: 0, 'Desconhecido'), uma medida estatística (média `.mean()`, mediana `.median()`, moda `.mode()[0]`), ou usar métodos de preenchimento como `method='ffill'` (propaga o último valor válido para frente) ou `method='bfill'` (propaga o próximo valor válido para trás). A escolha da estratégia de imputação é crucial e depende do contexto.
* **🍳 Receita (Código Pandas):**

In [17]:
# 1. Verificar onde estão os nulos
print("--- Contagem de Nulos por Coluna (Antes) ---")
print(df_vendas.isnull().sum())

# 2. Estratégia A: Preencher o nulo na quantidade com a mediana da coluna
mediana_qtd = df_vendas['quantidade'].median() # Mediana é mais robusta
print(f"\nA mediana da quantidade é: {mediana_qtd}")

df_vendas_filled = df_vendas.copy() # Criar cópia para não alterar o original
df_vendas_filled['quantidade'] = df_vendas_filled['quantidade'].fillna(mediana_qtd)
print("\n--- DataFrame com Nulos Preenchidos (fillna com mediana) ---")
print(df_vendas_filled)
print("Contagem de Nulos (Depois do fillna):")
print(df_vendas_filled['quantidade'].isnull().sum()) # Deve ser 0

# 3. Estratégia B: Remover a linha que contém nulos na coluna 'quantidade'
df_vendas_dropped = df_vendas.dropna(subset=['quantidade']) # Remove a linha da Mesa
print("\n--- DataFrame com Linha Nula Removida (dropna) ---")
print(df_vendas_dropped)
print(f"Número de linhas antes: {len(df_vendas)}, depois do dropna: {len(df_vendas_dropped)}")

--- Contagem de Nulos por Coluna (Antes) ---
id_venda          0
produto           0
categoria         0
quantidade        1
preco_unitario    0
id_cliente        0
data_venda        0
produto_limpo     0
dtype: int64

A mediana da quantidade é: 1.5

--- DataFrame com Nulos Preenchidos (fillna com mediana) ---
   id_venda          produto    categoria  quantidade  preco_unitario  \
0         1           Laptop  Eletrônicos         1.0          3500.0   
1         2          Teclado  Eletrônicos         2.0           150.0   
2         3           Caneca         Casa        10.0            25.0   
3         4          Monitor  Eletrônicos         1.0          1200.0   
4         5    Cadeira Gamer       Móveis         1.0           950.0   
5         6            Mouse  Eletrônicos         3.0            80.0   
6         7  Mesa de Escrit.       Móveis         1.5           450.0   

   id_cliente data_venda   produto_limpo  
0         101 2025-10-20          laptop  
1         102 202

* **📊 Resultado:** A primeira parte mostra que apenas a coluna `quantidade` tem um valor nulo. `df_vendas_filled` terá a linha 7 ('Mesa de Escrit.') com a `quantidade` preenchida pelo valor da mediana (que é 1.0 neste caso). `df_vendas_dropped` terá apenas 6 linhas, pois a linha 7 foi completamente removida. A estratégia `fillna` preserva mais dados, enquanto `dropna` é mais simples, mas pode enviesar a análise se muitos dados forem removidos. A escolha depende do percentual de dados faltantes e do impacto da variável.

## 3. Receitas Avançadas de Limpeza e Transformação

A limpeza básica (tipos, nulos, duplicatas) é só o começo. Frequentemente, precisamos transformar os dados para torná-los mais úteis ou padronizados.

### Receita 3.1: Limpeza Avançada de Texto (`.str` e `regex`)

* **🧠 Intuição:** "Os nomes dos produtos estão uma bagunça ('Laptop Modelo X!', 'Teclado (Sem Fio)'). Vamos tirar os caracteres especiais ('!') e padronizar, talvez extrair só a informação principal ('Laptop', 'Teclado')."
* **🎓 Definição Técnica:** Uso de métodos do acessador `.str` combinados com expressões regulares (`regex`) para limpar e padronizar strings. Inclui remoção de caracteres indesejados (`replace`), extração de padrões (`extract`), verificação de conteúdo (`contains`), e padronização de caixa (`lower`, `upper`, `title`).
* **🍳 Receita:**

In [16]:
# Exemplo: Limpar coluna 'produto'
df_vendas['produto_limpo'] = df_vendas['produto'].str.lower() # Padroniza caixa
# Remove caracteres não alfanuméricos (exceto espaço) usando regex
df_vendas['produto_limpo'] = df_vendas['produto_limpo'].str.replace(r'[^\w\s]+', '', regex=True)
# Remove espaços extras no início/fim
df_vendas['produto_limpo'] = df_vendas['produto_limpo'].str.strip()

print("\n--- Coluna 'produto' após limpeza de texto ---")
print(df_vendas[['produto', 'produto_limpo']])


--- Coluna 'produto' após limpeza de texto ---
           produto   produto_limpo
0           Laptop          laptop
1          Teclado         teclado
2           Caneca          caneca
3          Monitor         monitor
4    Cadeira Gamer   cadeira gamer
5            Mouse           mouse
6  Mesa de Escrit.  mesa de escrit


* **📊 Resultado:** A nova coluna `produto_limpo` conterá versões padronizadas ('laptop', 'teclado', 'mesa de escrit'), mais fáceis de agrupar ou usar em modelos. Regex é uma ferramenta poderosa (e complexa) essencial para limpeza de texto.

### Receita 3.2: Manipulação de Datas (`pd.to_datetime` e `.dt`)

* **🧠 Intuição:** "A coluna `data_venda` já é uma data, mas eu quero saber *apenas* o mês da venda, ou o dia da semana, ou quantos dias se passaram desde a venda."
* **🎓 Definição Técnica:** Após garantir que a coluna é do tipo `datetime` (usando `pd.to_datetime`), o acessador `.dt` permite extrair componentes da data/hora (ano `.dt.year`, mês `.dt.month`, dia `.dt.day`, dia da semana `.dt.dayofweek`, etc.) ou calcular durações (`datetime.now() - df['data_venda']`).
* **🍳 Receita:**

In [18]:
# Garante que a coluna é datetime (já fizemos na criação)
# df_vendas['data_venda'] = pd.to_datetime(df_vendas['data_venda'])

# Extrair componentes
df_vendas['mes_venda'] = df_vendas['data_venda'].dt.month
df_vendas['dia_semana_venda'] = df_vendas['data_venda'].dt.dayofweek # 0=Segunda, 6=Domingo

# Calcular tempo decorrido (ex: dias desde a venda até hoje)
hoje = pd.to_datetime('today').normalize() # Pega a data de hoje, sem as horas
df_vendas['dias_desde_venda'] = (hoje - df_vendas['data_venda']).dt.days

print("\n--- Colunas de data extraídas e calculadas ---")
print(df_vendas[['data_venda', 'mes_venda', 'dia_semana_venda', 'dias_desde_venda']].head())


--- Colunas de data extraídas e calculadas ---
  data_venda  mes_venda  dia_semana_venda  dias_desde_venda
0 2025-10-20         10                 0                 9
1 2025-10-21         10                 1                 8
2 2025-10-21         10                 1                 8
3 2025-10-22         10                 2                 7
4 2025-10-23         10                 3                 6


* **📊 Resultado:** O DataFrame agora tem colunas adicionais (`mes_venda`, `dia_semana_venda`, `dias_desde_venda`) que podem ser usadas como *features* em modelos de ML (Engenharia de Features) ou para análises sazonais.

### Receita 3.3: Detecção e Tratamento (Básico) de Outliers

* **🧠 Intuição:** "O gráfico Boxplot mostrou um salário 'lá em cima', muito diferente dos outros. Esse valor extremo (outlier) pode distorcer nossa média e talvez até o modelo. Precisamos decidir: ele é um erro de digitação (removemos/corrigimos) ou é real (mantemos, ou usamos técnicas robustas)?"
* **🎓 Definição Técnica:** Outliers são pontos de dados que diferem significativamente de outras observações. Podem ser erros ou valores genuinamente extremos. Métodos comuns de detecção incluem inspeção visual (Boxplot, Scatterplot) ou regras estatísticas (ex: Z-score > 3, ou fora de 1.5 * IQR dos quartis). O tratamento varia: remoção, correção (se for erro claro), transformação (ex: log) ou uso de modelos robustos a outliers (como a Mediana ou árvores de decisão).
* **🍳 Receita:**

In [19]:
# 1. Detecção usando IQR (método do Intervalo Interquartil)
Q1 = df_vendas['preco_unitario'].quantile(0.25)
Q3 = df_vendas['preco_unitario'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR # método de Tukey
limite_superior = Q3 + 1.5 * IQR

print(f"\n--- Detecção de Outliers (Preço Unitário) ---")
print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Limites para outliers: Abaixo de {limite_inferior:.2f} ou Acima de {limite_superior:.2f}")

# Identificar os outliers
outliers = df_vendas[
    (df_vendas['preco_unitario'] < limite_inferior) | (df_vendas['preco_unitario'] > limite_superior)
]
print("\nOutliers encontrados:")
if not outliers.empty:
    print(outliers)
else:
    print("Nenhum outlier encontrado pelos limites do IQR.")

# 2. Tratamento (Exemplo: Capping - Limitar ao valor máximo/mínimo aceitável)
# df_vendas['preco_unitario_capped'] = df_vendas['preco_unitario'].clip(lower=limite_inferior, upper=limite_superior)
# print("\nPreço após capping (limitação):")
# print(df_vendas['preco_unitario_capped'])
# NOTA: Capping/Remoção deve ser feito com MUITO cuidado. Geralmente é melhor manter.


--- Detecção de Outliers (Preço Unitário) ---
Q1: 115.0, Q3: 1075.0, IQR: 960.0
Limites para outliers: Abaixo de -1325.00 ou Acima de 2515.00

Outliers encontrados:
   id_venda produto    categoria  quantidade  preco_unitario  id_cliente  \
0         1  Laptop  Eletrônicos         1.0          3500.0         101   

  data_venda produto_limpo  mes_venda  dia_semana_venda  dias_desde_venda  
0 2025-10-20        laptop         10                 0                 9  


* **📊 Resultado:** O código calcula os limites baseados no IQR. No nosso exemplo, o Laptop (3500) provavelmente será identificado como outlier superior. A seção de tratamento (comentada) mostra como poderíamos "limitar" (cap) esse valor ao `limite_superior`, mas ressalta que a remoção ou alteração de outliers exige análise crítica do negócio.

### Receita 3.4: Binning (Discretização) com `pd.cut` e `pd.qcut`

* **🧠 Intuição:** "A coluna `preco_unitario` tem muitos valores diferentes (R$ 25, R$ 150, R$ 3500...). Isso pode ser muito detalhado para uma análise ou modelo. Vamos simplificar criando 'faixas de preço' (ex: 'Barato', 'Médio', 'Caro')."
* **🎓 Definição Técnica:** Processo de converter uma variável numérica contínua em uma variável categórica discreta (bins ou faixas). Isso pode reduzir o ruído e capturar relações não-lineares.
    * **`pd.cut` (Corte por Valor):** Você define os *limites* exatos das faixas (ex: 0-200, 201-1000, 1001+). Isso é bom quando você tem regras de negócio claras.
    * **`pd.qcut` (Corte por Quantil):** Você define *quantas faixas* quer (ex: 3), e o Pandas garante que cada faixa tenha (aproximadamente) o *mesmo número de produtos*. Isso é bom para segmentação.
* **🍳 Receita:**

In [20]:
# Usando o DataFrame deste exercício: df_vendas
print("--- Preços Originais ---")
print(df_vendas[['produto', 'preco_unitario']])

# --- Opção 1: pd.cut (Definindo os limites de valor) ---
# Queremos faixas: Barato (0-200), Médio (200-1000), Caro (1000+)
# Usamos -np.inf e np.inf para garantir que pegamos todos os valores
faixas_valor = [-np.inf, 200, 1000, np.inf]
nomes_faixas_valor = ['Barato', 'Médio', 'Caro']

# Criamos a nova coluna 'faixa_preco_valor'
df_vendas['faixa_preco_valor'] = pd.cut(
    df_vendas['preco_unitario'],
    bins=faixas_valor,
    labels=nomes_faixas_valor,
    right=True # right=True (padrão) significa que o intervalo inclui o limite direito (ex: (200, 1000])
)

print("\n--- Binning com pd.cut (Corte por Valor) ---")
print(df_vendas[['produto', 'preco_unitario', 'faixa_preco_valor']])


# --- Opção 2: pd.qcut (Dividindo em grupos de tamanho igual) ---
# Queremos 3 grupos (tercis), não importa o valor, apenas a quantidade
# q=3 significa 3 grupos (tercis)
nomes_faixas_quantil = ['Mais Barato (33%)', 'Intermediário (33%)', 'Mais Caro (33%)']

# Criamos a nova coluna 'faixa_preco_quantil'
df_vendas['faixa_preco_quantil'] = pd.qcut(
    df_vendas['preco_unitario'],
    q=3,
    labels=nomes_faixas_quantil
)

print("\n--- Binning com pd.qcut (Corte por Quantil) ---")
print(df_vendas[['produto', 'preco_unitario', 'faixa_preco_quantil']])

# Veja a contagem para provar que pd.qcut dividiu os grupos:
print("\nContagem por quantil (pd.qcut):")
print(df_vendas['faixa_preco_quantil'].value_counts())

--- Preços Originais ---
           produto  preco_unitario
0           Laptop          3500.0
1          Teclado           150.0
2           Caneca            25.0
3          Monitor          1200.0
4    Cadeira Gamer           950.0
5            Mouse            80.0
6  Mesa de Escrit.           450.0

--- Binning com pd.cut (Corte por Valor) ---
           produto  preco_unitario faixa_preco_valor
0           Laptop          3500.0              Caro
1          Teclado           150.0            Barato
2           Caneca            25.0            Barato
3          Monitor          1200.0              Caro
4    Cadeira Gamer           950.0             Médio
5            Mouse            80.0            Barato
6  Mesa de Escrit.           450.0             Médio

--- Binning com pd.qcut (Corte por Quantil) ---
           produto  preco_unitario  faixa_preco_quantil
0           Laptop          3500.0      Mais Caro (33%)
1          Teclado           150.0  Intermediário (33%)
2       

* **📊 Resultado:**
    * O DataFrame `df_vendas` agora tem duas novas colunas.
    * A coluna `faixa_preco_valor` (feita com `pd.cut`) mostra 'Barato' (Teclado, Caneca, Mouse), 'Médio' (Cadeira, Mesa) e 'Caro' (Laptop, Monitor), com base nos *valores* que definimos.
    * A coluna `faixa_preco_quantil` (feita com `pd.qcut`) dividiu os 7 produtos em 3 grupos com o número mais próximo possível de itens em cada (3, 2, 2). Veja como o 'Monitor' (1200) e 'Laptop' (3500) caíram no mesmo grupo 'Mais Caro (33%)'.
    * Esta técnica é uma poderosa ferramenta de **Engenharia de Features** (Pilar 1 do ML).